## NIR Tutorial 

In [ ]:
# Plot images in line 
%matplotlib inline
import matplotlib
# Set the desired figure size to get printed out 
matplotlib.rcParams["figure.figsize"] = (8.0, 8.0)
# Import PlantCV 
from plantcv import plantcv as pcv

In [ ]:
class options:
    def __init__(self):
        self.image = "img/tutorial_images/vis/original_image.jpg"
        self.debug = "plot"
        self.writeimg = False
        self.result = "vis_tutorial_results.txt"

In [ ]:
# Get options
args = options()

# Read image
img, path, filename = pcv.readimage(args.image, args.debug)

In [ ]:
# Read in the background image 
img_bkgrd = cv2.imread("background_nir_z2500.png", flags=0)

In [ ]:
# Subtract the background image from the image with the plant. 
bkg_sub_img = pcv.image_subtrat(img, img_bkgrd)

In [ ]:
# Threshold the image of interest using the two-sided cv2.inRange function (keep what is between 50-190) 
bkg_sub_thres_img = cv2.inRange(bkg_sub_img, 50, 190)
# Since we are using an OpenCV function, we need to make it print
if args.debug == 'print': 
    pcv.print_image(bkg_sub_thres_img, 'bkgrd_sub_thres.png')
elif args.debug == 'plot'
    pcv.plot_image(bkg_sub_thres_img)

In [ ]:
# Laplace filtering (identify edges based on 2nd derivative)
lp_img = pcv.laplace_filter(img, 1, 1)
if args.debug:
        pcv.plot_hist(lp_img, 'hist_lp')

In [ ]:
# Lapacian image sharpening, this step will enhance the darkness of the edges detected
lp_shrp_img = pcv.image_subtract(img, lp_img)
if args.debug:
        pcv.plot_hist(lp_sharp_img, 'hist_lp_sharp')

In [ ]:
# Sobel filtering
# 1st derivative sobel filtering along horizontal axis, kernel = 1)
sbx_img = pcv.sobel_filter(img, 1, 0, 1)

In [ ]:
# 1st derivative sobel filtering along vertical axis, kernel = 1)
sby_img = pcv.sobel_filter(img, 0, 1, 1)

In [ ]:
# Combine the effects of both x and y filters through matrix addition
# This will capture edges identified within each plane and emphesize edges found in both images
sb_img = pcv.image_add(sbx_img, sby_img)

In [ ]:
# Use a lowpass (blurring) filter to smooth sobel image
mblur_img = pcv.median_blur(sb_img, 1)

In [ ]:
mblur_invert_img = pcv.invert(mblur_img)

In [ ]:
# combine the smoothed sobel image with the laplacian sharpened image
# combines the best features of both methods as described in "Digital Image Processing" by Gonzalez and Woods pg. 169
edge_shrp_img = pcv.image_add(mblur_invert_img, lp_shrp_img)

In [ ]:
# Perform thresholding to generate a binary image
tr_es_img = pcv.threshold.binary(edge_shrp_img, 145, 255, 'dark')

In [ ]:
# Do erosion with a 3x3 kernel
e1_img = pcv.erode(tr_es_img, 3, 1)

In [ ]:
# Bring the two object identification approaches together.
# Using a logical OR combine object identified by background subtraction and the object identified by derivative filter.
comb_img = pcv.logical_or(e1_img, bkg_sub_thres_img)

In [ ]:
# Get masked image, Essentially identify pixels corresponding to plant and keep those.
masked_erd = pcv.apply_mask(img, comb_img, 'black')

In [ ]:
# Need to remove the edges of the image, we did that by generating a set of rectangles to mask the edges
# img is (254 X 320)
# Mask for the bottom of the image
masked1, box1_img, rect_contour1, hierarchy1 = pcv.rectangle_mask(img, (120,184), (215,252))

In [ ]:
# Mask for the left side of the image
masked2, box2_img, rect_contour2, hierarchy2 = pcv.rectangle_mask(img, (1,1), (85,252))

In [ ]:
# Mask for the right side of the image
masked3, box3_img, rect_contour3, hierarchy3 = pcv.rectangle_mask(img, (240,1), (318,252))

In [ ]:
# Mask the edges
masked4, box4_img, rect_contour4, hierarchy4 = pcv.rectangle_mask(img, (1,1), (318,252))

In [ ]:
# Combine boxes to filter the edges and car out of the photo
bx12_img = pcv.logical_or(box1_img, box2_img)

In [ ]:
bx123_img = pcv.logical_or(bx12_img, box3_img)

In [ ]:
bx1234_img = pcv.logical_or(bx123_img, box4_img)

In [ ]:
# Invert this mask and then apply it the masked image.
inv_bx1234_img = pcv.invert(bx1234_img)

In [ ]:
edge_masked_img = pcv.apply_mask(masked_erd, inv_bx1234_img, 'black')

In [ ]:
# Identify objects
id_objects,obj_hierarchy = pcv.find_objects(edge_masked_img, inv_bx1234_img)

In [ ]:
# Define ROI
roi1, roi_hierarchy= pcv.roi.rectangle(x=100, y=100, h=200, w=200, img=edge_masked_img)

In [ ]:
# Decide which objects to keep
roi_objects, hierarchy5, kept_mask, obj_area = pcv.roi_objects(edge_masked_img, 'partial', roi1, roi_hierarchy, id_objects, obj_hierarchy)

In [ ]:
# Use the object_composition function to outline the plant 
rgb_img = cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
o, m = pcv.object_composition(rgb_img, roi_objects, hierarchy5)

Now we can perform the analysis of pixelwise signal value and object shape attributes.


In [ ]:
### Analysis ###

outfile=False
if args.writeimg==True:
    outfile=args.outdir+"/"+filename

# Perform signal analysis
nir_header, nir_data, nir_img = pcv.analyze_nir_intensity(img, kept_mask, 256, args.outdir + '/' + img_name)
# Perform shape analysis
shape_header, shape_data, shape_img = pcv.analyze_object(rgb_img, o, m, args.outdir + '/' + img_name)

# Write shape and nir data to results file
result=open(args.result,"a")
result.write('\t'.join(map(str,shape_header)))
result.write("\n")
result.write('\t'.join(map(str,shape_data)))
result.write("\n")
for row in shape_img:
    result.write('\t'.join(map(str,row)))
    result.write("\n")
result.write('\t'.join(map(str,nir_header)))
result.write("\n")
result.write('\t'.join(map(str,nir_data)))
result.write("\n")
for row in nir_img:
    result.write('\t'.join(map(str,row)))
    result.write("\n")
result.close()